In [1]:
import os

os.chdir('../..')

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [4]:
import analysis.plotting as pl
from sklearn.metrics import accuracy_score

In [5]:
trial = 51

In [6]:
"""Script to calculate gait parameters from Kinect data."""

import os

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

import modules.gait_parameters as gp
import modules.numpy_funcs as nf


load_dir = os.path.join('data', 'kinect', 'best_pos')
save_dir = os.path.join('data', 'kinect', 'gait_params')
match_dir = os.path.join('data', 'matching')

df_match = pd.read_csv(os.path.join(match_dir, 'match_kinect_zeno.csv'))


load_total_dir = os.path.join('results', 'dataframes')
df_total_k = pd.read_pickle(os.path.join(load_total_dir, 'df_total_k.pkl'))
df_total_z = pd.read_pickle(os.path.join(load_total_dir, 'df_total_z.pkl'))

pass_counts_z = df_total_z.groupby(['trial_id']).nunique()['pass']
pass_counts_k = pd.Series(index=pass_counts_z.index)


for match_row in df_match.itertuples():
    
    trial_id, file_name = match_row.trial_id, match_row.kinect

    file_path = os.path.join(load_dir, file_name + '.pkl')

    df_head_feet = pd.read_pickle(file_path)
    frames = df_head_feet.index

    # Convert points to floats to allow future calculations (e.g. np.isnan)
    df_head_feet = df_head_feet.applymap(lambda point: point.astype(np.float))

    # Cluster frames with mean shift to locate the walking passes
    clustering = DBSCAN(eps=5).fit(nf.to_column(frames))
    labels = clustering.labels_
    
    pass_counts_k.loc[trial_id] = sum(np.unique(labels) >= 0)
    
    if trial_id == trial:
        break

In [7]:
# points = np.column_stack((frames, np.ones(len(frames))))
# pl.scatter_labels(points, labels)

In [8]:
df_match = df_match.set_index('trial_id')

In [9]:
head_points = np.stack(df_head_feet.HEAD)

In [10]:
plt.plot(head_points[:, 0])

In [11]:
# pass_counts_k[pass_counts_k != pass_counts_z]

In [12]:
# pass_counts_z[pass_counts_k != pass_counts_z]

In [13]:
pass_counts_k.loc[trial]

4.0

In [14]:
pass_counts_z.loc[trial]

6

In [16]:
df_match.loc[trial]

zeno      B_2014-12-22_P007_Pre_tw_D
kinect       2014-12-22_P007_Pre_003
Name: 51, dtype: object